In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

Please run make_data_structure.sh prior to this

In [2]:
filename = 'cities.json'

graph_cities = src.graph.graph_from_json(filename)

In [3]:
with open('Data/AFDC/evse_stations.json', 'r') as file:
    evse = json.load(file)

In [4]:
stations_raw = []

for station in evse['fuel_stations']:

    checks = (
        station['state'] == 'CA',
        station['ev_dc_fast_num'] is not None,
        station['access_code'] == 'public',
        station['status_code'] == 'E',
    )

    if all(checks):

        stations_raw.append(station)

In [5]:
df_stations = pd.DataFrame.from_dict(stations_raw)

In [6]:
node_attributes = {
    'x': 'lambda n: n["longitude"]',
    'y': 'lambda n: n["latitude"]',
    'n_dcfc': 'lambda n: n["ev_dc_fast_num"]',
    'n_acl2': 'lambda n: n["ev_level2_evse_num"]',
    'network': 'lambda n: n["ev_network"]',
    'name': 'lambda n: n["station_name"]',
    'address': 'lambda n: n["street_address"]',
    'city': 'lambda n: n["city"]',
    'state': 'lambda n: n["state"]',
    'zip': 'lambda n: n["zip"]',
    'access_code': 'lambda n: n["access_code"]',
    'status_code': 'lambda n: n["status_code"]',
    'type': 'lambda n: "station"',
}

nlg = src.graph.nlg_from_dataframe(df_stations, node_attributes)

graph_station = src.graph.graph_from_nlg(nlg)

mapping = {n: f'station_{n}' for n in graph_station.nodes}

graph_station = nx.relabel_nodes(graph_station, mapping)

In [7]:
graph_empty = nx.union(graph_cities, graph_station)

In [8]:
graph_empty.number_of_nodes(), graph_empty.number_of_edges()

(1921, 0)

In [10]:
src.graph.graph_to_json(graph_empty, 'Outputs/graph_empty.json')